# Извлечение навыков из поля `description`

## Импорт библиотек

In [2]:
import re
import pandas as pd
import numpy as np

from pathlib import Path
from itertools import chain

In [3]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


In [4]:
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

MODEL_ID = "ai-forever/sbert_large_nlu_ru"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(MODEL_ID, device=device)

model


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [5]:
# Хранилище обработанных данных:

PROCESSED_DIR = Path('../data/processed')
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


## Работа с данными

In [6]:
df = pd.read_csv("https://izemenkov.github.io/skillgrow/data/raw_data/vacancies_master.csv")

# Оставляем только описание
texts = df["description"].dropna().copy()

# Простейшая очистка
def clean_text(t):
    t = re.sub(r"<.*?>", " ", t)         # убираем html разметку, если сохранилась
    t = re.sub(r"[^а-яА-Яa-zA-Z0-9+#/\. ]", " ", t)  # убираем спецсимволы
    t = re.sub(r"\s+", " ", t)
    return t.lower().strip()

texts = texts.apply(clean_text)

df["clean_text"] = texts
df.head()

,id,name,area,experience,key_skills,description,clean_text
0,127202494,ML-аналитик / Data Scientist,Москва,Нет опыта,NaN,Обязанности: Моделирование клиентского поведе...,обязанности моделирование клиентского поведени...
1,127244399,Аналитик данных/Data Analyst,Нижний Новгород,Нет опыта,"Python, SQL, Базы данных, Анализ данных","Ищем специалиста, готового влиться в команду п...",ищем специалиста готового влиться в команду пр...
2,126209623,"Стажер, Data Analyst / Data Scientist",Москва,Нет опыта,"Python, pandas, Numpy, ООП, Алгоритмы и структ...",В сегодняшней бизнес-среде компаниям необходим...,в сегодняшней бизнес среде компаниям необходим...
3,127192646,Младший аналитик данных/Junior Data Analyst,Москва,От 1 года до 3 лет,"Python, SQL, Аналитическое мышление, Исследова...","Mediascope – исследовательская компания, котор...",mediascope исследовательская компания которая ...
4,126562421,Junior Data Analyst/Аналитик данных,Казань,Нет опыта,NaN,Привет! Мы команда разработчиков платформы You...,привет мы команда разработчиков платформы youn...


In [7]:
# Создадим список навыков
df["skills_list"] = df["key_skills"].fillna("").apply(lambda x: [s.strip().lower() for s in x.split(",") if s.strip()])

# объединяем все навыки в один список
all_skills = list(chain.from_iterable(df["skills_list"]))

skill_freq = (
    pd.Series(all_skills)
    .value_counts()
    .reset_index()
    .rename(columns={"index": "skill", 0: "count"})
)

skill_freq.head(20)


,skill,count
0,python,205
1,sql,151
2,pandas,50
3,pytorch,42
4,ml,40
5,анализ данных,39
6,big data,38
7,математическая статистика,31
8,numpy,28
9,визуализация данных,27


In [8]:
# Проверим на "шум"

# Навыки короче 2 символов
skill_freq[skill_freq.skill.str.len() < 2]

# Видим только r, оставим

,skill,count
66,r,5


In [9]:
# Навыки, встречающиеся менее 3 раз
skill_freq[skill_freq["count"] < 3]

,skill,count
94,vba,2
95,обучение и развитие,2
96,разработка по,2
97,power query,2
98,json,2
...,...,...
327,проактивность,1
328,qlikview,1
329,программирование,1
330,ии,1


Навыки, встречающиеся менее двух раз уберем. Среди них, визуально, есть и значащие, но, зачастую, они являются подразделами более частых навыков. Попробуем обойтись без них

In [10]:
# убираем навыки частотой < 2 и навык it, как незначащий
skill_vocab = skill_freq[(skill_freq["count"] >= 3) & (skill_freq["skill"] != 'it')]["skill"].tolist()
len(skill_vocab), skill_vocab[:30]


(93,
 ['python',
  'sql',
  'pandas',
  'pytorch',
  'ml',
  'анализ данных',
  'big data',
  'математическая статистика',
  'numpy',
  'визуализация данных',
  'power bi',
  'nlp',
  'hadoop',
  'machine learning',
  'deep learning',
  'data science',
  'tensorflow',
  'clickhouse',
  'mlflow',
  'git',
  'scikit-learn',
  'postgresql',
  'docker',
  'etl',
  'ms excel',
  'cv',
  'pyspark',
  'llm',
  'data analysis',
  'аналитическое мышление'])

In [11]:
# Извлекаем навыки с помощью полученного списка:

df["clean_text"] = df["description"].fillna("").str.lower()

def extract_from_text(text, vocab):
    results = []
    for s in vocab:
        if len(s) == 1:
            # skill состоит из 1 буквы: ищем только как отдельное слово
            # пример: r → ищем ` r ` или начало/конец строки
            pattern = r'\b' + re.escape(s) + r'\b'
            if re.search(pattern, text):
                results.append(s)

        else:
            # обычные навыки: разрешаем подстроки
            if s in text:
                results.append(s)

    return results

df["skills_from_description"] = df["clean_text"].apply(lambda t: extract_from_text(t, skill_vocab))


In [12]:
# Проверим покрытие
coverage = (df["skills_from_description"].apply(len) > 0).mean()
print(f"Доля вакансий с навыками из описания: {coverage:.1%}")

skills_flat = list(chain.from_iterable(df["skills_from_description"]))
pd.Series(skills_flat).value_counts().head(20)


Доля вакансий с навыками из описания: 99.8%


python          420
sql             344
ml              323
pandas          191
pytorch         151
numpy           148
llm             139
аналитика       131
nlp             128
git             124
spark           116
docker          106
api              99
scikit-learn     79
data science     76
power bi         71
clickhouse       71
etl              70
tableau          65
pyspark          65
Name: count, dtype: int64

Удалось добиться хорошего покрытия вакансий навыками.
Теперь приедем все к единому полю навыков, с которым будем работать в дальнейшем

In [13]:
def unite_skills(row):
    s1 = row.get("skills_list", [])
    s2 = row.get("skills_from_description", [])
    # объединяем, чистим и сортируем
    return sorted(set(map(str.lower, s1 + s2)))

df["final_skills"] = df.apply(unite_skills, axis=1)


In [14]:
# Смотрим частоты по финальному списку навыков

final_skills_flat = list(chain.from_iterable(df["final_skills"]))
pd.Series(final_skills_flat).value_counts().head(25)


python           438
sql              359
ml               325
pandas           202
pytorch          161
numpy            157
llm              140
git              136
аналитика        132
nlp              129
spark            121
docker           111
api               99
data science      88
scikit-learn      86
hadoop            79
power bi          77
clickhouse        77
анализ данных     76
pyspark           72
etl               71
tensorflow        68
matplotlib        67
tableau           66
big data          66
Name: count, dtype: int64

In [15]:
# Смотрим частотность навыков, исходя из опыта
exploded = df.explode("final_skills")
grp = exploded.groupby(["experience", "final_skills"]).size().reset_index(name="count")
grp.sort_values("count", ascending=False).groupby("experience").head(15)


,experience,final_skills,count
445,От 3 до 6 лет,python,236
466,От 3 до 6 лет,sql,190
415,От 3 до 6 лет,ml,185
230,От 1 года до 3 лет,python,147
242,От 1 года до 3 лет,sql,123
446,От 3 до 6 лет,pytorch,100
434,От 3 до 6 лет,pandas,98
200,От 1 года до 3 лет,ml,96
410,От 3 до 6 лет,llm,84
428,От 3 до 6 лет,numpy,80


### Выводы

* Удалось добиться извлечения навыков из вакансий с хорошим покрытием (>99%)
* Собрали ядро DS компетенций для рынка РФ
| Навык                                  | Частота                                                           | Комментарий                             |
| -------------------------------------- | ----------------------------------------------------------------- | --------------------------------------- |
| **python**                             | 407                                                               | базовый рабочий язык                    |
| **sql**                                | 333                                                               | must-have для аналитики и продакшена    |
| **ml**                                 | 306                                                               | обобщающее, но отражает роль            |
| **pandas / numpy**                     | 186 / 143                                                         | работа с табличными данными             |
| **pytorch / tensorflow**               | 150 / 62                                                          | модели в проде, особенно pytorch        |
| **nlp / llm**                          | 119 / 131                                                         | **всплеск из-за LLM тренда**, это важно |
| **spark / hadoop / pyspark**           | 115 / 76 / 71                                                     | Big Data → всё чаще must-have           |
| **docker / git / api**                 | 105 / 127 / 88                                                    | продакшен навыки                        |
| **power bi / clickhouse / postgresql** | Power BI = BI для бизнеса; ClickHouse/PostgreSQL                  | инфраструктура                       |


* Навыки против опыта показывают возможный карьерный трек специалиста:

| Опыт          | Ключевые навыки                                              | Интерпретация                                |
| ------------- | ------------------------------------------------------------ | -------------------------------------------- |
| **Нет опыта** | python, sql, pandas, power bi                                | Вход с BI / аналитики ↔ стажировки           |
| **1–3 года**  | python, sql, ml, pandas, nlp, numpy, docker                  | переход «анализ → модели → прототипирование» |
| **3–6 лет**   | python, sql, llm, pytorch, spark, docker, clickhouse, hadoop | продакшн ML + Big Data + архитектура         |
| **> 6 лет**   | python, sql, ml, big data, power bi, cv                      | роли "ведущий / архитектор / руководитель"   |


MVP Системы рекомендаций может работать следующим образом:

"Вы целитесь в позицию с опытом X. Средние навыки для этой зоны: ...
У вас есть: ...
Рекомендуем добрать: ..."


In [16]:
# Сохраним файл мастер-данных в ../data/processed
df_to_save = df[['id','name','area','experience','clean_text','final_skills']].rename(columns={'clean_text':'description'})

In [17]:
df_to_save

,id,name,area,experience,description,final_skills
0,127202494,ML-аналитик / Data Scientist,Москва,Нет опыта,обязанности: моделирование клиентского поведе...,"[etl, ml, mlflow, pandas, python, scikit-learn..."
1,127244399,Аналитик данных/Data Analyst,Нижний Новгород,Нет опыта,"ищем специалиста, готового влиться в команду п...","[dwh, python, sql, анализ данных, аналитика, б..."
2,126209623,"Стажер, Data Analyst / Data Scientist",Москва,Нет опыта,в сегодняшней бизнес-среде компаниям необходим...,"[big data, data science, git, ml, ms sql, nump..."
3,127192646,Младший аналитик данных/Junior Data Analyst,Москва,От 1 года до 3 лет,"mediascope – исследовательская компания, котор...","[data science, dwh, jupyter notebook, python, ..."
4,126562421,Junior Data Analyst/Аналитик данных,Казань,Нет опыта,привет! мы команда разработчиков платформы you...,"[numpy, pandas, power bi, python, sql, tableau]"
...,...,...,...,...,...,...
512,127525182,Data Analyst (Идентификация и трафик),Москва,От 3 до 6 лет,описание проекта: ит b2c — самая крупная экоси...,"[apache spark, python, spark, sql]"
513,127549143,Middle Data Analyst / DA (Медийная реклама),Москва,От 3 до 6 лет,мы ищем middle data analyst / data engineer в ...,"[clickhouse, etl, spark, sql]"
514,127499653,Эксперт (комплекс систем для управления data-п...,Иркутск,От 1 года до 3 лет,команда «apl витрины» дивизиона «розничное взы...,"[dwh, etl, greenplum, postgresql, sql]"
515,127549047,Data analyst middle (Медийная реклама),Москва,От 3 до 6 лет,ит b2c — самая крупная экосистема в сбере. нас...,"[python, sql, аналитика]"


In [18]:
df_to_save.to_csv(PROCESSED_DIR / 'extracted_skills.csv', index=False)

## Создание векторной базы вакансий (навыков и описаний)

In [19]:
df = df_to_save.copy(deep=True)

In [20]:
# Приведем к строкам поля описания и навыков

def list_to_text(x):
    if isinstance(x, (list, tuple, set)):
        return " ".join(map(str, x))
    if pd.isna(x):
        return ""
    return str(x)

df['final_skills'] = df['final_skills'].apply(list_to_text).fillna('')


df['description'] = df['description'].fillna("").astype(str)



In [21]:
def batched_encode(texts, batch_size=64, normalize=True):
    """Возвращает np.ndarray размера [N, D]."""
    texts = list(texts)  # на случай Series
    embs = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        with torch.inference_mode():
            e = model.encode(
                batch,
                convert_to_numpy=True,
                normalize_embeddings=normalize
            )
        embs.append(e)
    return np.vstack(embs) if embs else np.zeros((0, model.get_sentence_embedding_dimension()), dtype=np.float32)

In [22]:
df

,id,name,area,experience,description,final_skills
0,127202494,ML-аналитик / Data Scientist,Москва,Нет опыта,обязанности: моделирование клиентского поведе...,etl ml mlflow pandas python scikit-learn sql
1,127244399,Аналитик данных/Data Analyst,Нижний Новгород,Нет опыта,"ищем специалиста, готового влиться в команду п...",dwh python sql анализ данных аналитика базы да...
2,126209623,"Стажер, Data Analyst / Data Scientist",Москва,Нет опыта,в сегодняшней бизнес-среде компаниям необходим...,big data data science git ml ms sql numpy pand...
3,127192646,Младший аналитик данных/Junior Data Analyst,Москва,От 1 года до 3 лет,"mediascope – исследовательская компания, котор...",data science dwh jupyter notebook python spark...
4,126562421,Junior Data Analyst/Аналитик данных,Казань,Нет опыта,привет! мы команда разработчиков платформы you...,numpy pandas power bi python sql tableau
...,...,...,...,...,...,...
512,127525182,Data Analyst (Идентификация и трафик),Москва,От 3 до 6 лет,описание проекта: ит b2c — самая крупная экоси...,apache spark python spark sql
513,127549143,Middle Data Analyst / DA (Медийная реклама),Москва,От 3 до 6 лет,мы ищем middle data analyst / data engineer в ...,clickhouse etl spark sql
514,127499653,Эксперт (комплекс систем для управления data-п...,Иркутск,От 1 года до 3 лет,команда «apl витрины» дивизиона «розничное взы...,dwh etl greenplum postgresql sql
515,127549047,Data analyst middle (Медийная реклама),Москва,От 3 до 6 лет,ит b2c — самая крупная экосистема в сбере. нас...,python sql аналитика


In [23]:
skills_emb = batched_encode(df["final_skills"], batch_size=64)
desc_emb = batched_encode(df["description"], batch_size=64)


desc_emb.shape, skills_emb.shape  # (N, D), (N, D)


100%|██████████| 9/9 [00:18<00:00,  2.06s/it]


((517, 1024), (517, 1024))

In [24]:
out = pd.DataFrame({
    "id": df_to_save["id"].astype(str),
    "desc_emb":  list(map(lambda v: v.astype(float).tolist(), desc_emb)),
    "skills_emb":list(map(lambda v: v.astype(float).tolist(), skills_emb)),
})
out.to_parquet("../data/processed/embeddings.parquet", index=False)
